In [1]:
import pandas as pd
import data_cleaning as dc
import en_core_web_lg
model = en_core_web_lg.load()

['barcode']


In [2]:
df_model_answer = pd.read_excel("model_answer.xlsx", keep_default_na=False, na_values=["__"])

In [4]:
df_model_answer.shape

(3, 1)

In [5]:
def preprocess(df, input_col, out_col):
    print("preprocess cleaning")
    temp = df.copy()
    for index, row in temp.iterrows():
        raw_text = str(row[input_col])
        cleaned_text = dc.stop_and_stem(dc.text_clean(raw_text), stem = False)
        slist = cleaned_text.split()
        cleaned_text = " ".join(sorted(set(slist), key = slist.index))
        temp.loc[index, out_col] = cleaned_text
    return temp

In [6]:
input_col = "model_answer"
output_col = "model_answer_cleaned"
df_model_answer = preprocess(df_model_answer, input_col, output_col)

preprocess cleaning


In [7]:
df_model_answer

,model_answer,model_answer_cleaned
0,A barcode is a series of light and dark bars o...,series light dark bars differing widths
1,With a code number printed underneath,code number printed underneath
2,"Made up of country of origin code, manufacture...",made up country origin code manufacturer produ...


In [8]:
df_model_answer.to_csv('cleaning_check.csv')

In [9]:
df_student_answer = pd.read_excel("student_answer.xlsx", "Sheet1", keep_default_na=False, na_values=["__"])

In [10]:
df_student_answer.head()

,#,Student Answer,Marks,Justification,Complexity,Ref Answer
0,1,A barcode is a series of white and black bars ...,1,1,medium,1. A barcode is a series of light and dark bar...
1,2,A barcode is represnted by a sequence of light...,1,1,medium,
2,3,A barcode is a series of light and dark bars o...,1,1,medium,
3,4,Barcode is a kind of code with bars with a cod...,1,2,easy,
4,5,Barcode is a series of bars with a code number...,1,2,medium,


In [11]:
input_col = "Student Answer"
output_col = "student_answer_cleaned"
df_student_answer = preprocess(df_student_answer, input_col, output_col)

preprocess cleaning


In [12]:
df_student_answer[["Student Answer", "student_answer_cleaned"]].head()

,Student Answer,student_answer_cleaned
0,A barcode is a series of white and black bars ...,series white black bars differing widths paste...
1,A barcode is represnted by a sequence of light...,represnted sequence light dark bars different ...
2,A barcode is a series of light and dark bars o...,series light dark bars differing thickness
3,Barcode is a kind of code with bars with a cod...,kind code bars number printed underneath
4,Barcode is a series of bars with a code number...,series bars code number printed bottom


In [13]:
df_student_answer.to_csv('check_cleaning_1.csv')

In [14]:
def Similarity(original,compared):
    score = 0

    if str(original) == str(compared):
        score = 1
        return score
    elif len(original)==0 or len(compared)==0:
            score = 0
    else:
        word_vec_org = model(original)
        word_vec_com = model(compared)
        score = word_vec_org.similarity(word_vec_com)
    return score

In [32]:
answers = []
score_1 = []
score_2 = []
score_3 = []
actual_marks = []
reason = []

for index1, rows1 in df_student_answer.iterrows():
    answers.append(rows1['Student Answer'])
    actual_marks.append(rows1['Marks'])
    reason.append(rows1['Justification'])
    score_1.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][0]))
    score_2.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][1]))
    score_3.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][2]))

new = pd.DataFrame(list(zip(answers,score_1,score_2,score_3,actual_marks,reason)),
                   columns =['Answer','Score with point 1','Score with point 2','Score with point 3','Actual Marks', 'Justification'])
new.to_csv('output_answer_scores.csv', index = False)
    

In [26]:
text1 = "Man sleeps"
text2 = "Lady wakes"

Similarity(text1, text2)

0.6822474784200924

In [27]:
df_65 = pd.read_excel("65_answers.xlsx", "Consolidated", keep_default_na=False, na_values=["__"])

In [28]:
df_65.head()

,#,Student Answer,Marks,Justification,Complexity,Ref Answer
0,1,A barcode is a rectangular strip comprising of...,1,2,tough,1. A barcode is a series of light and dark bar...
1,2,A barcode is a bar with product code printed. ...,0,0,tough,
2,3,It is a machine readable numbers arranged in a...,1,1,tough,
3,4,Barcode is a unique series of parallel lines p...,1,1,tough,
4,5,Barcode uses a phone's camera to turn itself i...,0,0,easy,


In [29]:
input_col = "Student Answer"
output_col = "student_answer_cleaned"
df_65 = preprocess(df_65, input_col, output_col)

preprocess cleaning


In [31]:
#df_65[["Student Answer", "student_answer_cleaned"]]

In [46]:
def jaccard_similarity(text1, text2):
    s1 = set(text1.split())
    s2 = set(text2.split())
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [47]:
answers = []
score_1 = []
score_2 = []
score_3 = []
actual_marks = []
reason = []
jac_1 = []
jac_2 = []
jac_3 = []

for index1, rows1 in df_65.iterrows():
    answers.append(rows1['Student Answer'])
    actual_marks.append(rows1['Marks'])
    reason.append(rows1['Justification'])
    score_1.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][0]))
    score_2.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][1]))
    score_3.append(Similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][2]))
    jac_1.append(jaccard_similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][0]))
    jac_2.append(jaccard_similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][1]))
    jac_3.append(jaccard_similarity(rows1['student_answer_cleaned'],df_model_answer['model_answer_cleaned'][2]))

new = pd.DataFrame(list(zip(answers,score_1,score_2,score_3,actual_marks,reason,jac_1,jac_2,jac_3)),
                   columns =['Answer','Score with point 1','Score with point 2','Score with point 3','Actual Marks','Justification',
                            'Jac_1', 'Jac_2', 'Jac_3'])
new.to_csv('65_temp_scores.csv', index = False)

In [45]:
token1 = model("machine")
print(token1.vector)
token2 = model("learning")
print(token2.vector)
token3 = model("machine-learning")
print(token3.vector)

[-1.2538e-01  3.8888e-01  4.8011e-01 -4.9002e-01 -3.6593e-03 -2.8381e-01
 -3.0941e-01 -1.3498e-01  1.9084e-01  8.0288e-01  4.2266e-01 -2.9402e-01
 -2.6985e-01 -1.7700e-01 -3.0054e-01  1.7076e-02 -3.2872e-01  2.2793e+00
 -3.6797e-01  2.6255e-01  4.4370e-01 -8.0851e-02 -1.1741e-01  5.5017e-01
  1.6258e-01  5.4599e-02 -1.2735e-01 -1.5065e-01 -2.2314e-01 -2.2071e-01
 -5.1473e-02 -2.3058e-01 -7.6502e-02  4.3800e-01  2.2685e-01  1.5235e-01
 -3.8982e-01  2.6686e-01 -1.4056e-01 -3.1321e-02  2.7343e-01  7.4885e-01
 -6.5482e-01 -1.7675e-01 -4.4518e-01  3.2047e-01  8.0788e-02 -6.1032e-02
 -4.0233e-01  6.1603e-01  1.4369e-01  3.6725e-01 -6.7580e-01  4.1636e-02
 -1.8292e-01 -4.0382e-01  7.0964e-02 -2.4140e-01  1.0813e-01 -5.5925e-02
 -2.7747e-01 -2.1325e-02  3.7895e-02  2.6205e-01  4.8563e-01  1.4882e-01
  2.6676e-01  3.8100e-01 -5.1741e-01  3.8224e-01 -2.0476e-01 -3.9371e-01
 -1.4245e-01 -5.5597e-01  3.3155e-01  3.7561e-01  1.0290e-01 -1.4021e-01
 -3.4799e-02  5.4190e-01 -4.6772e-01 -7.5114e-01  4

  1.38040006e-01 -1.31446660e-01 -1.90599915e-02  2.96900004e-01]
